- 기업은행 관련 기사 (콜센터) 관련 내용에서 bert를 이용하여 키워드 추출하는 예제

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


특정 문서의 주요 정보를 이해하고자 할 때 키워드 추출을 통해서 입력 텍스트와 가장 관련이 있는 단어와 구문을 추출할 수 있습니다. 'Rake'나 'YAKE!'와 같은 키워드를 추출하는 데 사용할 수 있는 패키지가 이미 존재합니다. 그러나 이러한 모델은 일반적으로 텍스트의 통계적 특성에 기반하여 작동하며 의미적인 유사성에 대해서는 고려하지 않습니다. 의미적 유사성을 고려하기 위해서 여기서는 SBERT 임베딩을 활용하여 사용하기 쉬운 키워드 추출 알고리즘인 KeyBERT를 사용합니다.

# 1. 기본 KeyBERT

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
doc = """
IBK기업은행이 한국능률협회컨설팅(KMAC)에서 주관하는 '2022년 한국산업의 서비스 품질지수(KSQI)' 콜센터 부문에서 16년 연속 '한국의 우수콜센터'로 선정됐다.

기업은행은 '고객중심'의 고객센터를 실현하기 위해 △수신여건 개선 △AI(인공지능)기술 활용 △상담품질 개선 △상담직원 케어 등 다양한 노력을 기울였고 그 결과 16년 연속 우수콜센터 선정이라는 명예를 달성할 수 있었다.

먼저 코로나19 팬데믹으로 소상공인 지원대출 문의 증가, 비대면 전용상품 출시 증가 등 인입콜이 증가함에 따라 '수신여건' 고도화에 집중했다. 특정 업무로 집중되는 콜을 분산하기 위해 해당 서비스코드 전면 배치, 안내멘트 송출 등 ARS 시스템을 개선했고, 특정 이슈 발생 시 긴급 투입이 가능한 신속 대응팀도 신설·운영했다.

업무 생산성을 높이기 위해 기업뱅킹·퇴직연금·소상공인 대출 등 상담직원 간 순환근무 활성화로 전문적이고 복합적인 기업업무 문의에 대해 원스톱(ONE-STOP) 서비스를 실시했다.

또한 언택트 시대 도래로 비대면 채널 이용고객이 증가함에 따라 AI분석기법인 텍스트 마이닝과 딥러닝 기법을 활용해 고객센터로 유입되는 다양한 고객의 소리를 면밀히 분석했다. 이러한 분석 결과를 직원들에게 전파하고 실제 상담과정에 반영해 고객과의 공감과 소통에 역점을 두었다.

기업은행 콜센터는 상담 품질 개선에도 아낌없는 투자와 지원을 했다. 특히 다양하고 세분화된 고객 문의에 응대하기 위해 상담인력에 대한 단계적이고 체계적인 교육, 그리고 역량 강화에 힘썼다. 또한 다양한 비대면 교육 채널을 통한 전문지식 함양과 업무역량 향상에 많은 노력을 기울였다.

기업은행 관계자는 "앞으로도 기업은행 고객센터만의 '더 쉽고, 더 빠르게, 더 편리하게' 특화된 서비스를 제공하겠다"고 말했다.
      """

In [ ]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('IBK', 'Alpha'), ('기업은행', 'Noun'), ('이', 'Josa'), ('한국', 'Noun'), ('능률', 'Noun'), ('협회', 'Noun'), ('컨설팅', 'Noun'), ('(', 'Punctuation'), ('KMAC', 'Alpha')]
명사 추출 : 기업은행 한국 능률 협회 컨설팅 주관 한국 산업 서비스 품질 지수 콜센터 부문 연속 한국 우수 콜센터 로 선정 기업은행 고객 중심 의 고객 센터 실현 위해 수신 여건 개선 인공 지능 기술 활용 상담 품질 개선 상담 직원 케어 등 노력 그 결과 연속 우수 콜센터 선정 명예 달성 수 먼저 코로나 팬데믹 소상 공인 지원 대출 문의 증가 비대 전용 상품 출시 증가 등 인입콜 증가 함 수신 여건 고도화 집중 특정 업무 집중 콜 분산 위해 해당 서비스 코드 전면 배치 안내 멘트 송출 등 시스템 개선 특정 이슈 발생 시 긴급 투입 신속 대응 팀 신설 운영 업무 생산 위해 기업 뱅킹 퇴직 연금 소상 공인 대출 등 상담 직원 간 순환 근무 활성화 전문 복합 기업 업무 문의 대해 원스톱 서비스 실시 또한 언택트 시대 도래 비대 채널 이용 고객 증가 함 분석 법인 텍스트 마 이닝 딥 러닝 기법 활용 고객 센터 유입 고객 소리 분석 분석 결과 직원 전파 실제 상담 과정 반영 고객 공감 소통 역점 기업은행 콜센터 상담 품질 개선 투자 지원 세분 고객 문의 응대 위해 상담 인력 대한 단계 체계 교육 역량 강화 또한 비대 교육 채널 통한 전문 지식 함양 업무 역량 향상 노력 기업은행 관계자 앞 기업은행 고객 센터 더 더 더 특화된 서비스 제공 고 말


여기서는 사이킷런의 CountVectorizer를 사용하여 단어를 추출합니다.  CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 단쉽게 n-gram을 추출할 수 있기 때문입니다. 예를 들어, (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과 3개의 단어를 한 묶음으로 간주하는 trigram을 추출합니다.

In [ ]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 361
trigram 다섯개만 출력 : ['강화 또한' '강화 또한 비대' '개선 상담' '개선 상담 직원' '개선 인공']


다음으로 이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화 할 차례입니다. 한국어를 포함하고 있는 다국어 SBERT를 로드합니다.

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

이제 문서와 가장 유사한 키워드들을 추출합니다. 여기서는 문서와 가장 유사한 키워드들은 문서를 대표하기 위한 좋은 키워드라고 가정합니다. 상위 5개의 키워드를 출력합니다.

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [ ]:
keywords

['콜센터 선정 기업은행', '개선 투자 지원', '연속 우수 콜센터', '기업은행 한국 능률', '향상 노력 기업은행']

5개의 키워드가 출력되는데, 이들의 의미가 좀 비슷해보입니다. 비슷한 의미의 키워드들이 리턴되는 데는 이유가 있습니다. 당연히 이 키워드들이 문서를 가장 잘 나타내고 있기 때문입니다. 만약, 지금 출력한 것보다는 좀 더 다양한 의미의 키워드들이 출력된다면 이들을 그룹으로 본다는 관점에서는 어쩌면 해당 키워드들이 문서를 잘 나타낼 가능성이 적을 수도 있습니다. 따라서 키워드들을 다양하게 출력하고 싶다면 키워드 선정의 정확성과 키워드들의 다양성 사이의 미묘한 균형이 필요합니다.

여기서는 다양한 키워드들을 얻기 위해서 두 가지 알고리즘을 사용합니다.

* Max Sum Similarity  
* Maximal Marginal Relevance

# 2. Max Sum Similarity

데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

이를 위해 상위 10개의 키워드를 선택하고 이 10개 중에서 서로 가장 유사성이 낮은 5개를 선택합니다.

낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['한국 우수 콜센터', '투자 지원', '개선 투자', '고객 센터 실현', '기업은행 한국 능률']

그러나 상대적으로 높은 nr_candidates는 더 다양한 키워드 5개를 만듭니다.

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['역점 기업은행 콜센터', '상품 출시 증가', '품질 개선 투자', '기업은행 한국', '우수 콜센터 선정']

# 3. Maximal Marginal Relevance

결과를 다양화하는 마지막 방법은 MMR(Maximum Limit Relegance)입니다. MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다. 먼저 문서와 가장 유사한 키워드를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

만약 우리가 상대적으로 낮은 diversity 값을 설정한다면, 결과는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['향상 노력 기업은행', '연속 우수 콜센터', '기업은행 한국 능률', '고객 센터 실현', '개선 투자 지원']

그러나 상대적으로 높은 diversity값은 다양한 키워드 5개를 만들어냅니다.

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['향상 노력 기업은행', '연속 한국 우수', '인공 지능', '결과 직원 전파', '공감 소통 역점']